In [1]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install --upgrade --no-cache-dir gdown
!gdown --id 11hcgbRcs_9mV1QkiDq8Eb6FKENQClI2f
!pip install patool
import patoolib
patoolib.extract_archive('archive.zip')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0
/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=11hcgbRcs_9mV1QkiDq8Eb6FKENQClI2f
To: /content/archive.zip
100% 327M/327M [00:04<00:00, 79.2MB/s]
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 KB 2.8 MB/s eta 0:00:00
patool: Extracting archive.zip ...
patool: running /usr/bin/7z x -o./Unpack_f_rzdly5 -- archive.zip
patool: ... archive.zip extracted to `archive' (multiple files in root).


'archive'

In [3]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input, ZeroPadding2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionResNetV2

def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_inception_resnetv2_unet(input_shape):
    """ Input """
    inputs = Input(input_shape)

    """ Pre-trained InceptionResNetV2 Model """
    encoder = InceptionResNetV2(include_top=False, weights="imagenet", input_tensor=inputs)

    """ Encoder """
    s1 = encoder.get_layer("input_1").output           ## (512 x 512)

    s2 = encoder.get_layer("activation").output        ## (255 x 255)
    s2 = ZeroPadding2D(( (1, 0), (1, 0) ))(s2)         ## (256 x 256)

    s3 = encoder.get_layer("activation_3").output      ## (126 x 126)
    s3 = ZeroPadding2D((1, 1))(s3)                     ## (128 x 128)

    s4 = encoder.get_layer("activation_74").output      ## (61 x 61)
    s4 = ZeroPadding2D(( (2, 1),(2, 1) ))(s4)           ## (64 x 64)

    """ Bridge """
    b1 = encoder.get_layer("activation_161").output     ## (30 x 30)
    b1 = ZeroPadding2D((1, 1))(b1)                      ## (32 x 32)

    """ Decoder """
    d1 = decoder_block(b1, s4, 512)                     ## (64 x 64)
    d2 = decoder_block(d1, s3, 256)                     ## (128 x 128)
    d3 = decoder_block(d2, s2, 128)                     ## (256 x 256)
    d4 = decoder_block(d3, s1, 64)                      ## (512 x 512)

    """ Output """
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="InceptionResNetV2_U-Net")
    return model

if __name__ == "__main__":
    input_shape = (256, 256, 3)
    model = build_inception_resnetv2_unet(input_shape)
    model.summary()

219055592/219055592 [==============================] - 2s 0us/step
Model: "InceptionResNetV2_U-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 127, 127, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 127, 127, 32  96         ['conv2d[0][0]']                 
 alizatio

In [4]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.model_selection import train_test_split
import tensorflow as tf
from matplotlib import pyplot as plt
import keras
from keras.utils import normalize
from keras.metrics import MeanIoU
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
 

""" Global parameters """
H = 256
W = 256
def create_dir(path):
    """ Create a directory. """
    if not os.path.exists(path):
        os.makedirs(path)

def load_dataset(path, split=0.2):
    images = sorted(glob(os.path.join(path, "images", "*.png")))
    masks = sorted(glob(os.path.join(path, "masks", "*.png")))

    split_size = int(len(images) * split)

    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=split_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=split_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x / 255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (h, w)
    x = cv2.resize(x, (W, H))   ## (h, w)
    x = x / 255.0               ## (h, w)
    x = x.astype(np.float32)    ## (h, w)
    x = np.expand_dims(x, axis=-1)## (h, w, 1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset


In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [7]:
if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """
    create_dir("files")

    """ Hyperparameters """
    batch_size = 8
    lr = 1e-4
    num_epochs = 30
    model_path = os.path.join("files", "Final_model_inception_unet.h5")
    csv_path = os.path.join("files", "Final_model_inception_unet.csv")
    
    """ Dataset """
    dataset_path = "/content/archive"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(dataset_path)

    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")
    print(f"Test : {len(test_x)} - {len(test_y)}")
    train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

    """ Model """
    #model = build_inception_resnetv2_unet((H, W, 3))
    metrics = [dice_coef, iou, Recall(), Precision()]
    model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=["accuracy",metrics])

Train: 1840 - 1840
Valid: 612 - 612
Test : 612 - 612


In [8]:
    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path),
        EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=False),
    ]

    model.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=valid_dataset,
        callbacks=callbacks
    )

Epoch 1/30
230/230 [==============================] - ETA: 0s - loss: 0.7363 - accuracy: 0.9306 - dice_coef: 0.2637 - iou: 0.1569 - recall: 0.8831 - precision: 0.1890
Epoch 1: val_loss improved from inf to 0.79316, saving model to files/Final_model_inception_unet.h5
230/230 [==============================] - 148s 527ms/step - loss: 0.7363 - accuracy: 0.9306 - dice_coef: 0.2637 - iou: 0.1569 - recall: 0.8831 - precision: 0.1890 - val_loss: 0.7932 - val_accuracy: 0.9794 - val_dice_coef: 0.2067 - val_iou: 0.1163 - val_recall: 0.5813 - val_precision: 0.4470 - lr: 1.0000e-04
Epoch 2/30
230/230 [==============================] - ETA: 0s - loss: 0.5289 - accuracy: 0.9907 - dice_coef: 0.4711 - iou: 0.3126 - recall: 0.8830 - precision: 0.7042
Epoch 2: val_loss improved from 0.79316 to 0.67209, saving model to files/Final_model_inception_unet.h5
230/230 [==============================] - 121s 527ms/step - loss: 0.5289 - accuracy: 0.9907 - dice_coef: 0.4711 - iou: 0.3126 - recall: 0.8830 - precis

In [11]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
 
if __name__ == "__main__":
 
    """ Loading model """
    with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
        model = tf.keras.models.load_model("/content/files/Final_model_inception_unet.h5")

    """ Predicting the mask """
    create_dir("OutputImg")
    for x, y  in tqdm(zip(test_x,test_y), total=len(test_x)):
        """ Extracing the image name. """
        image_name = x.split("/")[-1]

        """ Reading the image """
        img = cv2.imread(x, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (W, H))
        x = img/255.0
        x = x.astype(np.float32)
        x = np.expand_dims(x, axis=0)
        
        img_y = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
        img_y = cv2.resize(img_y, (W, H))
        img_y = np.expand_dims(img_y, axis=-1)  ## (512, 512, 1)
        img_y = np.concatenate([img_y, img_y, img_y], axis=-1)  ## (512, 512, 3)

        """ Predicting the mask. """
        y_pred = model.predict(x)[0] > 0.5
        y_pred = y_pred.astype(np.int32)

        """ Saving the predicted mask along with the image and GT """
        save_image_path = f"/content/OutputImg/{image_name}"
        y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)

        sep_line = np.ones((H, 10, 3)) * 255

        output_image = np.concatenate([img, sep_line, img_y, sep_line, y_pred*255], axis=1)
        #cat_image = np.concatenate([sep_line,y_pred*255,sep_line], axis=1) # this line for just the output segment
        #cat_image = np.concatenate([ori_x,sep_line,y_pred*255], axis=1) # this line for just the output segment and the test image
        cv2.imwrite(save_image_path, output_image)

  0%|          | 0/612 [00:00<?, ?it/s]

1/1 [==============================] - 3s 3s/step


  0%|          | 1/612 [00:03<36:04,  3.54s/it]

1/1 [==============================] - 0s 33ms/step


  0%|          | 3/612 [00:03<10:07,  1.00it/s]

...

100%|█████████▉| 610/612 [01:13<00:00,  9.01it/s]

1/1 [==============================] - 0s 34ms/step


100%|█████████▉| 611/612 [01:13<00:00,  9.11it/s]

1/1 [==============================] - 0s 35ms/step


100%|██████████| 612/612 [01:13<00:00,  8.28it/s]


In [12]:
!zip -r /content/OutputImg.zip  /content/OutputImg 

  adding: content/OutputImg/ (stored 0%)
  adding: content/OutputImg/1585.png (deflated 16%)
  adding: content/OutputImg/351.png (deflated 17%)
  adding: content/OutputImg/1282.png (deflated 18%)
  adding: content/OutputImg/2854.png (deflated 16%)
  adding: content/OutputImg/1475.png (deflated 16%)
  adding: content/OutputImg/1927.png (deflated 22%)
  adding: content/OutputImg/2101.png (deflated 18%)
  adding: content/OutputImg/985.png (deflated 17%)
  adding: content/OutputImg/870.png (deflated 18%)
  adding: content/OutputImg/387.png (deflated 18%)
  adding: content/OutputImg/2110.png (deflated 20%)
  adding: content/OutputImg/1185.png (deflated 17%)
  adding: content/OutputImg/36.png (deflated 17%)
  adding: content/OutputImg/1304.png (deflated 17%)
  adding: content/OutputImg/1523.png (deflated 17%)
  adding: content/OutputImg/2216.png (deflated 20%)
  adding: content/OutputImg/3056.png (deflated 20%)
  adding: content/OutputImg/922.png (deflated 17%)
  adding: content/OutputImg/15